# Chapter7

## 예시 코드: TCP 기반 송/수신 게임
1. 숫자를 추측하는 게임을 위한 **TCP 기반 서버**
2. 이 서버는 고려할 **숫자의 범위를 결정하는 하한값과 상한값을 인자**로 받습니다.
3. 클라이언트가 요청하는 대로 그 범위 내의 정수 값에 대한 **추측을 반환**
4. 서버는 클라이언트로부터 각 숫자가 클라이언트의 비밀 숫자에 얼마나 가까웠는지(따뜻한지) 또는 멀어졌는지(차가운지)에 대한 **보고서를 수집**

***- Blocking I/O***

***- 데이터 전송이나 수신이 완료될 때까지 호출한 스레드가 대기(socket 사용의 특징 중 하나)***

### 송/수신 연결 구조

In [29]:
# python 3.8 이상에서만 실행됨(walus연산자때문)

class EOFError(Exception):
    pass

class ConnectionBase:
    def __init__(self, connection):
        self.connection = connection
        self.file = connection.makefile('rb')

    def send(self, command):
        line = command + '\n'
        data = line.encode()
        self.connection.send(data)

    def receive(self):
        line = self.file.readline()
        if not line:
            raise EOFError('연결 닫힘')
        return line[:-1].decode()



### 하나의 게임 구성(session)

In [30]:
import random

WARMER = '더따뜻함'
COLDER = '더차가움'
UNSURE = '잘모르겠음'
CORRECT = '맞음'

class UnknownCommandError(Exception):
    pass

class Session(ConnectionBase):
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state(None, None)

    def _clear_state(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.secret = None
        self.guesses = []

    def loop(self):
        while command := self.receive():
            parts = command.split(' ')
            if parts[0] == 'PARAMS':
                self.set_params(parts)
            elif parts[0] == 'NUMBER':
                self.send_number()
            elif parts[0] == 'REPORT':
                self.receive_report(parts)
            else:
                raise UnknownCommandError(command)

    def set_params(self, parts):    # 서버가 추축하려는 숫자의 범위를 설정하는 함수
        assert len(parts) == 3
        lower = int(parts[1])
        upper = int(parts[2])
        self._clear_state(lower, upper)

    def next_guess(self):   # 다음 추측할 숫자를 반환하는 함수
        if self.secret is not None:
            return self.secret

        while True:
            guess = random.randint(self.lower, self.upper)
            if guess not in self.guesses:
                return guess

    def send_number(self):  # 다음 추측할 숫자를 서버에 전송하는 함수
        guess = self.next_guess()
        self.guesses.append(guess)
        self.send(format(guess))

    def receive_report(self, parts):    # 서버로부터 결과를 받아오는 함수
        assert len(parts) == 2
        decision = parts[1]
        last = self.guesses[-1]
        if decision == CORRECT:
            self.secret = last

        print(f'서버: {last}는 {decision}')


### 클라이언트

In [31]:
import contextlib
import math

class Client(ConnectionBase):
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state() # 상태 초기화

    def _clear_state(self): # 상태 초기화
        self.secret = None
        self.last_distance = None

    @contextlib.contextmanager  # 컨텍스트 관리자로서, 게임의 시작과 끝에서 상태를 설정하고 초기화
    def session(self, lower, upper, secret):
        print(f'\n{lower}와 {upper} 사이의 숫자를 맞춰보세요!'
              f' 쉿! 그 숫자는 {secret} 입니다.')
        self.secret = secret
        self.send(f'PARAMS {lower} {upper}')    # 서버에게 숫자 범위를 알림
        try:
            yield   # 세션을 실행
        finally:
            self._clear_state() # 세션 종료 후 상태 초기화
            self.send('PARAMS 0 -1')    # 서버에게 숫자 범위를 초기화

    def request_numbers(self, count):   # 서버로부터 숫자를 요청하는 함수
        for _ in range(count):
            self.send('NUMBER') # 서버에게 숫자를 요청
            data = self.receive()   # 서버로부터 숫자를 받음
            yield int(data) # 숫자를 반환
            if self.last_distance == 0:
                return

    def report_outcome(self, number):   # 서버에게 결과를 전달하는 함수
        new_distance = math.fabs(number - self.secret)
        decision = UNSURE

        if new_distance == 0:
            decision = CORRECT
        elif self.last_distance is None:
            pass
        elif new_distance < self.last_distance:
            decision = WARMER
        elif new_distance > self.last_distance:
            decision = COLDER

        self.last_distance = new_distance

        self.send(f'REPORT {decision}')
        return decision



### socket

In [32]:
import socket
from threading import Thread


def handle_connection(connection):  # 클라이언트의 연결을 처리하는 함수
    with connection:
        session = Session(connection)
        try:
            session.loop()
        except EOFError:
            pass


def run_server(address):    # 서버를 실행하는 함수
    with socket.socket() as listener:   # 소켓 생성
        listener.bind(address)
        listener.listen()   # 클라이언트의 연결을 대기
        while True:
            connection, _ = listener.accept()   # 클라이언트의 연결이 들어올때마다 새로운 thread를 생성
            thread = Thread(target=handle_connection,
                            args=(connection,),
                            daemon=True)
            thread.start()

def run_client(address):
    with socket.create_connection(address) as connection:
        client = Client(connection)

        with client.session(1, 5, 3):
            results = [(x, client.report_outcome(x))
                       for x in client.request_numbers(5)]

        with client.session(10, 15, 12):
            for number in client.request_numbers(5):
                outcome = client.report_outcome(number)
                results.append((number, outcome))

    return results


def main():
    address = ('127.0.0.1', 3333)
    server_thread = Thread(
        target=run_server, args=(address,), daemon=True)
    server_thread.start()

    results = run_client(address)
    for number, outcome in results:
        print(f'클라이언트: {number}는 {outcome}')

main()


1와 5 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 3 입니다.
서버: 3는 맞음

10와 15 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 12 입니다.
서버: 15는 잘모르겠음
서버: 14는 더따뜻함
서버: 13는 더따뜻함
서버: 12는 맞음
클라이언트: 3는 맞음
클라이언트: 15는 잘모르겠음
클라이언트: 14는 더따뜻함
클라이언트: 13는 더따뜻함
클라이언트: 12는 맞음


## Item 61: Know How to Port Threaded I/O to asyncio

- 실제 통신 루프에 asyncio 적용

### 송/수신 연결 구조
- async def 와 await를 사용하여 비동기 기능 구현

In [33]:
# python 3.8 이상에서만 실행됨(walus연산자때문)

class EOFError(Exception):
    pass

class AsyncConnectionBase:
    def __init__(self, reader, writer):  # 변경됨
        self.reader = reader  # 변경됨
        self.writer = writer  # 변경됨

    async def send(self, command):
        line = command + '\n'
        data = line.encode()
        self.writer.write(data)  # 변경됨
        await self.writer.drain()  # 변경됨

    async def receive(self):
        line = await self.reader.readline()  # 변경됨
        if not line:
            raise EOFError('연결 닫힘')
        return line[:-1].decode()


### 하나의 게임 구성(session)
- AsyncConnectionBase를 상속받아 비동기 메서드 사용 가능

In [34]:
import random

WARMER = '더따뜻함'
COLDER = '더차가움'
UNSURE = '잘모르겠음'
CORRECT = '맞음'

class UnknownCommandError(Exception):
    pass

class AsyncSession(AsyncConnectionBase):  # 변경됨
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state(None, None)

    def _clear_state(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.secret = None
        self.guesses = []

    async def loop(self):  # 변경됨
        while command := await self.receive():  # 변경됨
            parts = command.split(' ')
            if parts[0] == 'PARAMS':
                self.set_params(parts)
            elif parts[0] == 'NUMBER':
                await self.send_number()  # 변경됨
            elif parts[0] == 'REPORT':
                self.receive_report(parts)
            else:
                raise UnknownCommandError(command)

    def set_params(self, parts):
        assert len(parts) == 3
        lower = int(parts[1])
        upper = int(parts[2])
        self._clear_state(lower, upper)

    def next_guess(self):
        if self.secret is not None:
            return self.secret

        while True:
            guess = random.randint(self.lower, self.upper)
            if guess not in self.guesses:
                return guess

    async def send_number(self):
        guess = self.next_guess()
        self.guesses.append(guess)
        await self.send(format(guess))

    def receive_report(self, parts):
        assert len(parts) == 2
        decision = parts[1]
        last = self.guesses[-1]
        if decision == CORRECT:
            self.secret = last

        print(f'서버: {last}는 {decision}')



### 클라이언트
- asynccontextmanager를 사용하여 비동기 컨텍스트를 관리

In [35]:

import contextlib
import math

class AsyncClient(AsyncConnectionBase):
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state()

    def _clear_state(self):
        self.secret = None
        self.last_distance = None

    @contextlib.asynccontextmanager                         # 변경됨
    async def session(self, lower, upper, secret):          # 변경됨
        print(f'\n{lower}와 {upper} 사이의 숫자를 맞춰보세요!'
              f' 쉿! 그 숫자는 {secret} 입니다.')
        self.secret = secret
        await self.send(f'PARAMS {lower} {upper}')          # 변경됨
        try:
            yield
        finally:
            self._clear_state()
            await self.send('PARAMS 0 -1')                   # 변경됨

    async def request_numbers(self, count):            # 변경됨
        for _ in range(count):
            await self.send('NUMBER')                  # 변경됨
            data = await self.receive()                # 변경됨
            yield int(data)
            if self.last_distance == 0:
                return

    async def report_outcome(self, number):                    # 변경됨
        new_distance = math.fabs(number - self.secret)
        decision = UNSURE

        if new_distance == 0:
            decision = CORRECT
        elif self.last_distance is None:
            pass
        elif new_distance < self.last_distance:
            decision = WARMER
        elif new_distance > self.last_distance:
            decision = COLDER

        self.last_distance = new_distance

        await self.send(f'REPORT {decision}')                  # 변경됨

        # 잠시 대기해서 출력 순서 조정
        await asyncio.sleep(0.01)
        return decision


### 실행
- 소켓 구성 없이 병렬적으로 실행가능

In [37]:
# Jupyter Notebook과 같은 환경에서는 기본적으로 이벤트 루프가 이미 실행 중인 상태. 이 경우, nest_asyncio 라이브러리를 사용
import nest_asyncio
nest_asyncio.apply()
# Jupyter Notebook이 아닌 경우
import asyncio

async def handle_async_connection(reader, writer):
    session = AsyncSession(reader, writer)
    try:
        await session.loop()
    except EOFError:
        pass

async def run_async_server(address):
    server = await asyncio.start_server(
        handle_async_connection, *address)
    async with server:
        await server.serve_forever()

async def run_async_client(address):
    # 서버가 시작될 수 있게 기다려주기
    await asyncio.sleep(0.1)

    streams = await asyncio.open_connection(*address)   # New
    client = AsyncClient(*streams)                      # New

    async with client.session(1, 5, 3):
        results = [(x, await client.report_outcome(x))
                   async for x in client.request_numbers(5)]

    async with client.session(10, 15, 12):
        async for number in client.request_numbers(5):
            outcome = await client.report_outcome(number)
            results.append((number, outcome))

    _, writer = streams                                # 새 기능
    writer.close()                                     # 새 기능
    await writer.wait_closed()                         # 새 기능

    return results

async def main_async():
    address = ('127.0.0.1', 4444)

    server = run_async_server(address)
    asyncio.create_task(server)

    results = await run_async_client(address)
    for number, outcome in results:
        print(f'클라이언트: {number}는 {outcome}')

asyncio.run(main_async())



1와 5 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 3 입니다.
서버: 1는 잘모르겠음
서버: 3는 맞음

10와 15 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 12 입니다.
서버: 15는 잘모르겠음
서버: 11는 더따뜻함
서버: 10는 더차가움
서버: 14는 잘모르겠음
서버: 13는 더따뜻함
클라이언트: 1는 잘모르겠음
클라이언트: 3는 맞음
클라이언트: 15는 잘모르겠음
클라이언트: 11는 더따뜻함
클라이언트: 10는 더차가움
클라이언트: 14는 잘모르겠음
클라이언트: 13는 더따뜻함
